## 1- Cargamos los datos

In [125]:
import pandas as pd

data = pd.read_csv("../datasets/converted_data/customer_dataset.csv", sep = ",")
data

,fraudulent,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer,transactionAmount,transactionFailed,orderAmount,orderState,emailDomain,emailProvider,customerIPAddressSimplified,sameCity
0,False,True,card,JCB 16 digit,Citizens First Banks,18,False,18.0,pending,com,yahoo,only_numeric_digits,yes
1,True,True,bitcoin,VISA 16 digit,Solace Banks,26,False,26.0,fulfilled,com,yahoo,only_numeric_digits,no
2,False,False,card,VISA 16 digit,Vertex Bancorp,45,False,45.0,fulfilled,com,yahoo,digits_and_letters,no
3,False,False,card,Diners Club / Carte Blanche,His Majesty Bank Corp.,23,False,23.0,fulfilled,com,yahoo,digits_and_letters,no
4,True,False,card,Mastercard,Vertex Bancorp,43,True,43.0,fulfilled,com,other,only_numeric_digits,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,NaN,NaN,NaN,NaN,NaN,25,True,NaN,NaN,weird,weird,only_numeric_digits,unknown
619,NaN,NaN,NaN,NaN,NaN,25,True,NaN,NaN,weird,weird,only_numeric_digits,unknown
620,NaN,NaN,NaN,NaN,NaN,25,False,NaN,NaN,weird,weird,only_numeric_digits,unknown
621,NaN,NaN,NaN,NaN,NaN,19,False,NaN,NaN,weird,weird,only_numeric_digits,unknown


In [126]:
from funpymodeling.exploratory import status

status(data)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,fraudulent,455,0.730337,107,0.171750,2,object
1,paymentMethodRegistrationFailure,281,0.451043,302,0.484751,2,object
2,paymentMethodType,281,0.451043,0,0.000000,4,object
3,paymentMethodProvider,281,0.451043,0,0.000000,10,object
4,paymentMethodIssuer,281,0.451043,0,0.000000,20,object
5,transactionAmount,0,0.000000,0,0.000000,67,int64
6,transactionFailed,0,0.000000,455,0.730337,2,bool
7,orderAmount,145,0.232745,0,0.000000,67,float64
8,orderState,145,0.232745,0,0.000000,3,object
9,emailDomain,0,0.000000,0,0.000000,6,object


## 2- Discretización

Hacemos una copia del dataframe __data__ para luego evaluar si los bins se guardan y leen correctamente.

In [127]:
probar = data.copy()

### 2.1- Discretizar variables

#### `transactionAmount`

In [128]:
data["transactionAmount"], transactionAmount_bins = pd.qcut(data["transactionAmount"], q = 4, duplicates = "drop", retbins = True)

#### `orderAmount`

In [129]:
data["orderAmount"], orderAmount_bins = pd.qcut(data["orderAmount"], q = 5, duplicates = "drop", retbins = True)

### 2.2 - Guardar bins y evaluarlos

Guardamos los bins generados por separado usando el modulo __pickle__.

In [130]:
import pickle

with open("../saved_bins/orderAmount_bins.pickle", "wb") as handle:
    pickle.dump(orderAmount_bins, handle, protocol = pickle.HIGHEST_PROTOCOL)

with open("../saved_bins/transactionAmount_bins.pickle", "wb") as handle:
    pickle.dump(transactionAmount_bins, handle, protocol = pickle.HIGHEST_PROTOCOL)

Luego de guardarlos, los evaluamos para corroborar que se hayan guardado correctamente.

In [131]:
with open("../saved_bins/orderAmount_bins.pickle", "rb") as handle:
    new_orderAmount_bins = pickle.load(handle)

probar["orderAmount"] = pd.cut(probar["orderAmount"],
                                bins = new_orderAmount_bins,
                                include_lowest = True)

probar["orderAmount"].head(5)

0    (9.999, 18.4]
1     (18.4, 30.0]
2     (39.0, 47.0]
3     (18.4, 30.0]
4     (39.0, 47.0]
Name: orderAmount, dtype: category
Categories (5, interval[float64, right]): [(9.999, 18.4] < (18.4, 30.0] < (30.0, 39.0] < (39.0, 47.0] < (47.0, 353.0]]

In [132]:
with open("../saved_bins/transactionAmount_bins.pickle", "rb") as handle:
    new_transactionAmount_bins = pickle.load(handle)

probar["transactionAmount"] = pd.cut(probar["transactionAmount"],
                                bins = new_transactionAmount_bins,
                                include_lowest = True)

probar["transactionAmount"].head(5)

0    (9.999, 21.0]
1     (21.0, 34.0]
2     (34.0, 45.0]
3     (21.0, 34.0]
4     (34.0, 45.0]
Name: transactionAmount, dtype: category
Categories (4, interval[float64, right]): [(9.999, 21.0] < (21.0, 34.0] < (34.0, 45.0] < (45.0, 353.0]]

## 3- Preparación de datos

### 3.1- Interpretación valores

#### `paymentMethodIssuer`

In [133]:
data["paymentMethodIssuer"].value_counts()

paymentMethodIssuer
Her Majesty Trust           43
Vertex Bancorp              37
Fountain Financial Inc.     35
His Majesty Bank Corp.      33
Bastion Banks               29
Bulwark Trust Corp.         29
Citizens First Banks        28
Grand Credit Corporation    27
Solace Banks                27
Rose Bancshares             25
B                            7
e                            5
c                            4
r                            3
                             2
n                            2
x                            2
o                            2
a                            1
p                            1
Name: count, dtype: int64

Como podemos observar varios métodos de pago raros, vamos a clasificarlos como _weird_.

In [134]:
weird_payment_method = ["B", "e", "c", "r", " ", "n", "x", "o", "a", "p"]

for payment_method in weird_payment_method:
    data["paymentMethodIssuer"] = data["paymentMethodIssuer"].replace(payment_method, "weird")

In [135]:
data["paymentMethodIssuer"].value_counts()

paymentMethodIssuer
Her Majesty Trust           43
Vertex Bancorp              37
Fountain Financial Inc.     35
His Majesty Bank Corp.      33
Bastion Banks               29
weird                       29
Bulwark Trust Corp.         29
Citizens First Banks        28
Solace Banks                27
Grand Credit Corporation    27
Rose Bancshares             25
Name: count, dtype: int64

#### `paymentMethodProvider`

In [136]:
data["paymentMethodProvider"].value_counts()

paymentMethodProvider
JCB 16 digit                   65
VISA 16 digit                  57
Voyager                        36
Diners Club / Carte Blanche    34
Maestro                        32
VISA 13 digit                  32
Discover                       25
American Express               22
JCB 15 digit                   20
Mastercard                     19
Name: count, dtype: int64

#### `paymentMethodType`

In [137]:
data["paymentMethodType"].value_counts()

paymentMethodType
card         242
apple pay     36
paypal        36
bitcoin       28
Name: count, dtype: int64

#### `fraudulent`

In [138]:
status(data).query("q_nan > 0")

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,fraudulent,455,0.730337,107,0.171750,2,object
1,paymentMethodRegistrationFailure,281,0.451043,302,0.484751,2,object
2,paymentMethodType,281,0.451043,0,0.000000,4,object
3,paymentMethodProvider,281,0.451043,0,0.000000,10,object
4,paymentMethodIssuer,281,0.451043,0,0.000000,11,object
7,orderAmount,145,0.232745,0,0.000000,5,category
8,orderState,145,0.232745,0,0.000000,3,object


Como la variable que vamos a predecir posee nulos, los vamos a reemplazar por _warning_.

In [139]:
data["fraudulent"] = data["fraudulent"].fillna(value = "warning")

In [140]:
data["fraudulent"].value_counts()

fraudulent
warning    455
False      107
True        61
Name: count, dtype: int64

Convertimos los valores a __str__ para asegurarnos de que no exista conflictos entre str y bool.

In [141]:
data["fraudulent"] = data["fraudulent"].astype("str")

Le asignamos un valor numérico a cada valor para hacer una especie de semáforo.
- _False_ = Le asignamos el valor numérico _0_, representaría el __verde__ del semáforo.
- _True_ = Le asignamos el valor numérico _1_, representaría el __rojo__ del semáforo.
- _warning_ = Le asignamos el valor numérico _2_, representaría el __amarillo__ del semáforo.

In [142]:
class_map = {"False":0, "True":1, "warning":2}
data["fraudulent"] = data["fraudulent"].map(class_map)

In [143]:
data.head(5)

,fraudulent,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer,transactionAmount,transactionFailed,orderAmount,orderState,emailDomain,emailProvider,customerIPAddressSimplified,sameCity
0,0,True,card,JCB 16 digit,Citizens First Banks,"(9.999, 21.0]",False,"(9.999, 18.4]",pending,com,yahoo,only_numeric_digits,yes
1,1,True,bitcoin,VISA 16 digit,Solace Banks,"(21.0, 34.0]",False,"(18.4, 30.0]",fulfilled,com,yahoo,only_numeric_digits,no
2,0,False,card,VISA 16 digit,Vertex Bancorp,"(34.0, 45.0]",False,"(39.0, 47.0]",fulfilled,com,yahoo,digits_and_letters,no
3,0,False,card,Diners Club / Carte Blanche,His Majesty Bank Corp.,"(21.0, 34.0]",False,"(18.4, 30.0]",fulfilled,com,yahoo,digits_and_letters,no
4,1,False,card,Mastercard,Vertex Bancorp,"(34.0, 45.0]",True,"(39.0, 47.0]",fulfilled,com,other,only_numeric_digits,no


### 3.2- Tratamiento de nulos

Lo hacemos para nuestras columnas categóricas, y en este caso, la única columna categórica con datos faltantes es `orderAmount`

In [144]:
data["orderAmount"].value_counts()

orderAmount
(30.0, 39.0]     98
(39.0, 47.0]     97
(9.999, 18.4]    96
(18.4, 30.0]     96
(47.0, 353.0]    91
Name: count, dtype: int64

In [145]:
data["orderAmount"] = data["orderAmount"].cat.add_categories("desconocido")
data["orderAmount"] = data["orderAmount"].fillna(value="desconocido")

In [146]:
data["orderAmount"].value_counts()

orderAmount
desconocido      145
(30.0, 39.0]      98
(39.0, 47.0]      97
(9.999, 18.4]     96
(18.4, 30.0]      96
(47.0, 353.0]     91
Name: count, dtype: int64

## 4- One Hot Encoding

In [147]:
data_ohe = pd.get_dummies(data)
data_ohe

,fraudulent,transactionFailed,paymentMethodRegistrationFailure_False,paymentMethodRegistrationFailure_True,paymentMethodType_apple pay,paymentMethodType_bitcoin,paymentMethodType_card,paymentMethodType_paypal,paymentMethodProvider_American Express,paymentMethodProvider_Diners Club / Carte Blanche,...,emailProvider_gmail,emailProvider_hotmail,emailProvider_other,emailProvider_weird,emailProvider_yahoo,customerIPAddressSimplified_digits_and_letters,customerIPAddressSimplified_only_numeric_digits,sameCity_no,sameCity_unknown,sameCity_yes
0,0,False,False,True,False,False,True,False,False,False,...,False,False,False,False,True,False,True,False,False,True
1,1,False,False,True,False,True,False,False,False,False,...,False,False,False,False,True,False,True,True,False,False
2,0,False,True,False,False,False,True,False,False,False,...,False,False,False,False,True,True,False,True,False,False
3,0,False,True,False,False,False,True,False,False,True,...,False,False,False,False,True,True,False,True,False,False
4,1,True,True,False,False,False,True,False,False,False,...,False,False,True,False,False,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,2,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,False,True,False
619,2,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,False,True,False
620,2,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,False,True,False
621,2,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,False,True,False


Eliminamos la variable objetivo de nuestro DataFrame para no guardarlo en nuestro archivo pickle, es decir, sin `fraudulent`.

In [148]:
data_ohe_without_fraudulent = data_ohe.drop(["fraudulent"], axis = 1)
data_ohe_without_fraudulent

,transactionFailed,paymentMethodRegistrationFailure_False,paymentMethodRegistrationFailure_True,paymentMethodType_apple pay,paymentMethodType_bitcoin,paymentMethodType_card,paymentMethodType_paypal,paymentMethodProvider_American Express,paymentMethodProvider_Diners Club / Carte Blanche,paymentMethodProvider_Discover,...,emailProvider_gmail,emailProvider_hotmail,emailProvider_other,emailProvider_weird,emailProvider_yahoo,customerIPAddressSimplified_digits_and_letters,customerIPAddressSimplified_only_numeric_digits,sameCity_no,sameCity_unknown,sameCity_yes
0,False,False,True,False,False,True,False,False,False,False,...,False,False,False,False,True,False,True,False,False,True
1,False,False,True,False,True,False,False,False,False,False,...,False,False,False,False,True,False,True,True,False,False
2,False,True,False,False,False,True,False,False,False,False,...,False,False,False,False,True,True,False,True,False,False
3,False,True,False,False,False,True,False,False,True,False,...,False,False,False,False,True,True,False,True,False,False
4,True,True,False,False,False,True,False,False,False,False,...,False,False,True,False,False,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,True,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,False,True,False
619,True,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,False,True,False
620,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,False,True,False
621,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,False,True,False


### 4.1 - Guardar ohe categories

In [149]:
with open("../ohe_categories_without_fraudulent.pickle", "wb") as handle:
    pickle.dump(data_ohe_without_fraudulent.columns, handle, protocol = pickle.HIGHEST_PROTOCOL)

## 5- Guardar dataset

In [150]:
filename = "../datasets/processed_data/ohe_customer_dataset.csv"
data_ohe.to_csv(filename, index = False)